In [1]:
import os
import cv2 as cv
import numpy as np
from datetime import datetime

In [2]:
def get_coordinates(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDBLCLK:  # Saat mouse double-click
        print(f"Mouse Hovering at: ({x}, {y})")

In [3]:
dir_project = "C:/Users/eats/projects/IASS"

In [36]:
name_video = "delta"
path_video = f"asset/{name_video}.mp4"
path_video = os.path.join(dir_project, path_video)

cap = cv.VideoCapture(path_video)
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)


### Get Coordinate and Crop Video

In [6]:
success, imx = cap.read()

# Buat window dan set event callback
cv.namedWindow("Image", cv.WINDOW_NORMAL)
cv.namedWindow("Image")
cv.setMouseCallback("Image", get_coordinates)

while True:
    cv.imshow("Image", imx)
    if cv.waitKey(1) & 0xFF == 27:  # Tekan ESC untuk keluar
        break

cv.destroyAllWindows()

In [15]:
imx.shape

(1080, 1920, 3)

Square Mask

In [37]:
# Line: (275, 249) -> (588, 260)
# Crop: (240, 291) -> (608, 291) -> (608, 99) -> (240, 99)
# (485, 507) and (693, 139)
x_min, x_max = 422, 1600
y_min, y_max = 140, 510

In [22]:
imx_crop = imx[y_min:y_max, x_min:x_max]

In [23]:
cv.imshow("imx_crop", imx_crop)
cv.waitKey(0)

-1

Get coordinates from cropped image

In [29]:
# Buat window dan set event callback
cv.namedWindow("Image", cv.WINDOW_NORMAL)
cv.namedWindow("Image")
cv.setMouseCallback("Image", get_coordinates)

while True:
    cv.imshow("Image", imx_crop)
    if cv.waitKey(1) & 0xFF == 27:  # Tekan ESC untuk keluar
        break

cv.destroyAllWindows()

Mouse Hovering at: (273, 0)
Mouse Hovering at: (950, 3)


In [24]:
imx_crop.shape

(370, 1920, 3)

Polygon Mask

In [ ]:
# polygon = np.array([(110, 1078), (801, 0), (0, 0), (0, 1078), (110, 1078)])

In [ ]:
# # Create a mask the same size as the image, initially all white (or copy original image)
# masked_img = imx.copy()

# # Draw filled black polygon on the image directly (in-place)
# cv.fillPoly(masked_img, [polygon], color=(0, 0, 0))  # Black color for mask

# cv.namedWindow("masked_img", cv.WINDOW_NORMAL)
# cv.imshow("masked_img", masked_img)
# cv.waitKey(0)

-1

Polygon Region

In [30]:
imx_crop.shape

(370, 1178, 3)

In [38]:
polygon = np.array([(0, imx_crop.shape[0]), (273, 0), (950, 0), (imx_crop.shape[1], imx_crop.shape[0])])

In [32]:
# Create a black mask (same height & width as image, single channel)
mask = np.zeros(imx_crop.shape[:2], dtype=np.uint8)

# Fill the polygon area with white (255)
cv.fillPoly(mask, [polygon], color=255)

# Create a mask the same size as the image, initially all white (or copy original image)
masked_img = imx_crop.copy()

# Apply the mask to keep only the polygon area
masked_img = cv.bitwise_and(masked_img, masked_img, mask=mask)

# # Draw filled black polygon on the image directly (in-place)
# cv.fillPoly(masked_img, [polygon], color=(0, 0, 0))  # Black color for mask

cv.namedWindow("masked_img", cv.WINDOW_NORMAL)
cv.imshow("masked_img", masked_img)
cv.waitKey(0)

-1

In [28]:
cv.waitKey(0)

-1

### View Cropped Video

In [ ]:
# # Video writer
# video_writer = cv.VideoWriter(f"{name_video}.avi", cv.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

In [39]:
name_datetime = str(datetime.now().date()).replace("-", "")
dir_output = os.path.join(dir_project, "asset", "dataset", name_datetime)
os.makedirs(dir_output, exist_ok=True)

# cv.namedWindow("Video", cv.WINDOW_NORMAL)

idx_img = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    # # imm = np.zeros_like(im0)
    # # imm[y_min:y_max, x_min:x_max] = im0[y_min:y_max, x_min:x_max]
    # imm = im0[y_min:y_max, x_min:x_max]
    
    # crop image
    imm = im0[y_min:y_max, x_min:x_max]

    # Create a black mask (same height & width as image, single channel)
    mask = np.zeros(imm.shape[:2], dtype=np.uint8)

    # Fill the polygon area with white (255)
    cv.fillPoly(mask, [polygon], color=255)

    # Apply the mask to keep only the polygon area
    imm = cv.bitwise_and(imm, imm, mask=mask)
    
    cv.imshow("Video", imm)
    if cv.waitKey(1) & 0xFF == ord("q"):
        break
    
    # video_writer.write(imm)
    
    path_output = os.path.join(dir_output, f'{name_video}_{idx_img}.jpg')
    cv.imwrite(path_output, imm)
    
    idx_img += 1
    
    # break

cap.release()           # Release the capture
# video_writer.release()  # Release the video writer
cv.destroyAllWindows()  # Destroy all windows

Video frame is empty or video processing has been successfully completed.


In [ ]:
# cv.imshow("im0", im0)
# cv.waitKey(0)

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:973: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
